# Final Project - Data Analysis

- INFO 5871-001
- Keke Wu
- Fall 2019

In [1]:
import pandas as pd
import numpy as np 
import matplotlib

### Import Datasets

Make dataframes for both csv files and drop the duplicates in demographics by user_id .

In [7]:
cc_df = pd.read_csv('cc.csv')
cc_df.head()

,user_id,vis_type,model_type,color_ramp_index,target_value,selected_value,aesthetic_rating,trial_number,age,occupation,gender,normal_vision,cdv,hours_per_day_on_computer,education,display_type,display_size_inches,design_experience,comments
0,2,heatmap,designer,1,6,6,0,1,30,Service Designer,Male,yes,no,6,graduateStudies,phone,4,8,NaN
1,2,scatter,designer,8,5,3,-20,2,30,Service Designer,Male,yes,no,6,graduateStudies,phone,4,8,NaN
2,2,choropleth,linear,21,8,8,27,3,30,Service Designer,Male,yes,no,6,graduateStudies,phone,4,8,NaN
3,2,choropleth,designer,11,6,6,24,4,30,Service Designer,Male,yes,no,6,graduateStudies,phone,4,8,NaN
4,2,scatter,bayesian,24,4,3,37,5,30,Service Designer,Male,yes,no,6,graduateStudies,phone,4,8,NaN


In [16]:
cc_df.groupby('model_type')